In [18]:
import json
import datetime
import time, os
import re

import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

from pymongo import MongoClient

from dotenv import load_dotenv
load_dotenv()

from tqdm.notebook import tqdm

In [3]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By

# chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver" # path to the chromedriver executable
# os.environ["webdriver.chrome.driver"] = chromedriver

In [19]:
API_KEY = os.getenv('THEMOVIEDB_API_KEY')

In [5]:
show_df = pd.DataFrame()
show_df['show'] = ['Westworld','Game of Thrones','The Flash'] #Brooklyn Nine-Nine
show_df['show_id'] = ['63247','1399','60735'] #48891
show_df['subreddit'] = ['westworld','gameofthrones','flashtv'] #brooklynninenine
show_df['wikipedia'] = ["Westworld_(TV_series)","List_of_Game_of_Thrones_episodes","List_of_The_Flash_episodes"] #List_of_Brooklyn_Nine-Nine_episodes
show_df['tomato'] = ["westworld","game_of_thrones","the_flash"] #brooklyn_nine_nine
show_df

,show,show_id,subreddit,wikipedia,tomato
0,Westworld,63247,westworld,Westworld_(TV_series),westworld
1,Game of Thrones,1399,gameofthrones,List_of_Game_of_Thrones_episodes,game_of_thrones
2,The Flash,60735,flashtv,List_of_The_Flash_episodes,the_flash


In [6]:
# wiki_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_Game_of_Thrones_episodes",attrs = {'class':'wikiepisodetable'})
# wiki_df[0]

In [7]:
def get_tomato_rating(show, season, ep = None):
    season = "s{:02d}".format(season)
    time.sleep(.5)
    if ep is None:
        url = f'https://www.rottentomatoes.com/tv/{show}/{season}'
        r = requests.get(url)
        page = r.text
        soup = BeautifulSoup(page, 'lxml')
        ep_item = soup.find(id = 'episode-list-root').find_all(class_ = 'episodeItem')
        
        return [span.find(class_ = 'tMeterScore').text if span.find(class_ = 'tMeterScore') is not None else None for span in ep_item]
    #[span.text for span in soup.find(id = 'episode-list-root').find_all(class_ = 'tMeterScore')]
    ep = "e{:02d}".format(ep)
    url = f'https://www.rottentomatoes.com/tv/{show}/{season}/{ep}'
    r = requests.get(url)
    page = r.text
    soup = BeautifulSoup(page, 'lxml')
    tomatos = soup.find(class_ = 'mop-ratings-wrap__percentage')
    if tomatos is not None:
        return soup.find(class_ = 'mop-ratings-wrap__percentage').text.strip()
    return None
get_tomato_rating('the_flash',6)

['100%', '100%', '78%', '88%', '88%', '100%', '88%', '78%', '100%', None]

In [8]:
client = MongoClient()
db = client.tv
show_info = db['show_info']
!mongodump -d tv

2020-07-02T10:17:36.834-0500	writing tv.tv to 
2020-07-02T10:17:36.886-0500	done dumping tv.tv (234 documents)
2020-07-02T10:17:37.138-0500	writing tv.posts to 
2020-07-02T10:17:37.140-0500	writing tv.comments to 
2020-07-02T10:17:37.667-0500	done dumping tv.posts (22974 documents)
2020-07-02T10:17:37.903-0500	done dumping tv.comments (46800 documents)


In [9]:
client.list_database_names()
list(db.tv.find({'subreddit': 'westworld'} ,{'_id':0, 'name':1 , 'episode_number': 1, 'tomato':1})) # ,{'_id':0, 'name':1 , 'episode_number': 1}

[{'episode_number': 1, 'name': 'The Original', 'tomato': '100%'},
 {'episode_number': 2, 'name': 'Chestnut', 'tomato': '96%'},
 {'episode_number': 3, 'name': 'The Stray', 'tomato': '90%'},
 {'episode_number': 4, 'name': 'Dissonance Theory', 'tomato': '96%'},
 {'episode_number': 5, 'name': 'Contrapasso', 'tomato': '93%'},
 {'episode_number': 6, 'name': 'The Adversary', 'tomato': '89%'},
 {'episode_number': 7, 'name': "Trompe L'Oeil", 'tomato': '96%'},
 {'episode_number': 8, 'name': 'Trace Decay', 'tomato': '95%'},
 {'episode_number': 9, 'name': 'The Well-Tempered Clavier', 'tomato': '86%'},
 {'episode_number': 10, 'name': 'The Bicameral Mind', 'tomato': '94%'},
 {'episode_number': 1, 'name': 'Journey Into Night', 'tomato': '87%'},
 {'episode_number': 2, 'name': 'Reunion', 'tomato': '95%'},
 {'episode_number': 3, 'name': 'Virtù e Fortuna', 'tomato': '100%'},
 {'episode_number': 4, 'name': 'The Riddle of the Sphinx', 'tomato': '95%'},
 {'episode_number': 5, 'name': 'Akane No Mai', 'tomato

In [186]:
db.tv.drop()
for show in show_df.itertuples():
    url = f'https://api.themoviedb.org/3/tv/{show[2]}?api_key={API_KEY}'
    r = requests.get(url)
    print(show[1])
    for season in json.loads(r.text)['seasons']:
        if season['season_number']:
            url = f'https://api.themoviedb.org/3/tv/{show[2]}/season/{season["season_number"]}?api_key={API_KEY}'
            r = requests.get(url)
            episodes = json.loads(r.text)['episodes']
            [ ep.pop('crew', None) for ep in episodes]
            [ ep.pop('guest_stars', None) for ep in episodes]
            tomato = get_tomato_rating(show[5],season['season_number'])
            for ep_index in range(len(episodes)):
                episodes[ep_index]['wikipedia'] = show[4]
                episodes[ep_index]['subreddit'] = show[3]
                if ep_index < 10:
                    if tomato[ep_index] is not None:
                        episodes[ep_index]['tomato'] = tomato[ep_index]
                else:
                    ep_tomato = get_tomato_rating(show[5],season['season_number'],ep_index+1)
                    if ep_tomato is not None:
                        episodes[ep_index]['tomato'] = ep_tomato
            db.tv.insert_many(episodes)
            print([ep.get('tomato') for ep in episodes])
#             print('air_date: ' + episodes['air_date'] + ", vote_average: "+ episodes['vote_average'])

Westworld
['100%', '96%', '90%', '96%', '93%', '89%', '96%', '95%', '86%', '94%']
['87%', '95%', '100%', '95%', '90%', '97%', '88%', '95%', '79%', '73%']
['90%', '89%', '94%', '84%', '68%', '84%', '53%', '50%']
Game of Thrones
['100%', '100%', '86%', '100%', '95%', '100%', '100%', '100%', '100%', '100%']
['100%', '89%', '100%', '97%', '96%', '100%', '96%', '100%', '100%', '92%']
['98%', '89%', '92%', '100%', '100%', '83%', '78%', '97%', '100%', '100%']
['95%', '100%', '96%', '96%', '98%', '96%', '100%', '96%', '94%', '98%']
['96%', '96%', '100%', '96%', '94%', '54%', '84%', '100%', '83%', '92%']
['86%', '88%', '87%', '100%', '98%', '88%', '98%', '84%', '98%', '99%']
['93%', '96%', '90%', '97%', '95%', '84%', '88%']
['92%', '87%', '73%', '58%', '49%', '48%']
The Flash
['90%', '90%', '90%', '100%', '90%', '75%', '90%', '100%', '84%', '100%', '83%', '80%', '78%', '90%', '90%', '85%', '95%', '77%', '80%', '100%', '100%', '86%', '100%']
['89%', '100%', '93%', '86%', '100%', '93%', '100%', '

In [13]:
list(db.tv.find({'subreddit': 'gameofthrones'} ,{'_id':0, 'subreddit':1 , 'air_date': 1, 'tomato': 1})) #

[{'air_date': '2011-04-17', 'subreddit': 'gameofthrones', 'tomato': '100%'},
 {'air_date': '2011-04-24', 'subreddit': 'gameofthrones', 'tomato': '100%'},
 {'air_date': '2011-05-01', 'subreddit': 'gameofthrones', 'tomato': '86%'},
 {'air_date': '2011-05-08', 'subreddit': 'gameofthrones', 'tomato': '100%'},
 {'air_date': '2011-05-15', 'subreddit': 'gameofthrones', 'tomato': '95%'},
 {'air_date': '2011-05-22', 'subreddit': 'gameofthrones', 'tomato': '100%'},
 {'air_date': '2011-05-29', 'subreddit': 'gameofthrones', 'tomato': '100%'},
 {'air_date': '2011-06-05', 'subreddit': 'gameofthrones', 'tomato': '100%'},
 {'air_date': '2011-06-12', 'subreddit': 'gameofthrones', 'tomato': '100%'},
 {'air_date': '2011-06-19', 'subreddit': 'gameofthrones', 'tomato': '100%'},
 {'air_date': '2012-04-01', 'subreddit': 'gameofthrones', 'tomato': '100%'},
 {'air_date': '2012-04-08', 'subreddit': 'gameofthrones', 'tomato': '89%'},
 {'air_date': '2012-04-15', 'subreddit': 'gameofthrones', 'tomato': '100%'},
 {

In [14]:
def get_reddit_posts(subreddit, air_date, endpoint = 'submission', limit = 100,):
    after = int((datetime.datetime.strptime(air_date,  "%Y-%m-%d") + datetime.timedelta(days=1)).timestamp())
    before = int((datetime.datetime.strptime(air_date,  "%Y-%m-%d") + datetime.timedelta(days=3)).timestamp())
    #  https://pushshift.io/api-parameters/
    parameters = [f'subreddit={subreddit}',
                 f'before={before}',
                 f'after={after}',
                 'sort_type=score',
                 'sort=desc',
                 f'limit={limit}',
#                  'score=>100'
                ]
    url = f'https://api.pushshift.io/reddit/{endpoint}/search/?' + '&'.join(parameters)
    r = requests.get(url)
    max_tries  = 1
    while r.status_code != 200:
        r = requests.get(url)
        time.sleep(1*max_tries**2)
        print(max_tries)
        if max_tries > 5:
            return {}
        max_tries += 1
    return json.loads(r.text)['data']
# [ str(posts['score']) + "   " + str(posts['num_comments']) for posts in get_reddit_posts('westworld','2018-04-22') ]

In [15]:
# r = requests.get("https://api.pushshift.io/reddit/submission/search/?subreddit=westworld")
# r.status_code

In [16]:
db.tv.count_documents({})

234

In [813]:
db.posts.drop()
#'subreddit': 'westworld'
for episode in list(db.tv.find({} ,{'_id':0, 'subreddit':1,
                                    'air_date': 1,
                                    'episode_number' : 1,
                                    'season_number': 1,
                                    'name': 1,
                                    'tomato': 1,
                                   })):
    print(episode)
    time.sleep(.1)
    posts = get_reddit_posts(episode['subreddit'],episode['air_date'])
    if posts:
        [ post.update({'episode': episode}) for post in posts ]
        db.posts.insert_many(posts)

{'air_date': '2016-10-02', 'episode_number': 1, 'name': 'The Original', 'season_number': 1, 'subreddit': 'westworld', 'tomato': '100%'}
{'air_date': '2016-10-09', 'episode_number': 2, 'name': 'Chestnut', 'season_number': 1, 'subreddit': 'westworld', 'tomato': '96%'}
{'air_date': '2016-10-16', 'episode_number': 3, 'name': 'The Stray', 'season_number': 1, 'subreddit': 'westworld', 'tomato': '90%'}
{'air_date': '2016-10-23', 'episode_number': 4, 'name': 'Dissonance Theory', 'season_number': 1, 'subreddit': 'westworld', 'tomato': '96%'}
{'air_date': '2016-10-30', 'episode_number': 5, 'name': 'Contrapasso', 'season_number': 1, 'subreddit': 'westworld', 'tomato': '93%'}
{'air_date': '2016-11-06', 'episode_number': 6, 'name': 'The Adversary', 'season_number': 1, 'subreddit': 'westworld', 'tomato': '89%'}
{'air_date': '2016-11-13', 'episode_number': 7, 'name': "Trompe L'Oeil", 'season_number': 1, 'subreddit': 'westworld', 'tomato': '96%'}
{'air_date': '2016-11-20', 'episode_number': 8, 'name':

{'air_date': '2014-04-13', 'episode_number': 2, 'name': 'The Lion and the Rose', 'season_number': 4, 'subreddit': 'gameofthrones', 'tomato': '100%'}
{'air_date': '2014-04-20', 'episode_number': 3, 'name': 'Breaker of Chains', 'season_number': 4, 'subreddit': 'gameofthrones', 'tomato': '96%'}
{'air_date': '2014-04-27', 'episode_number': 4, 'name': 'Oathkeeper', 'season_number': 4, 'subreddit': 'gameofthrones', 'tomato': '96%'}
{'air_date': '2014-05-04', 'episode_number': 5, 'name': 'First of His Name', 'season_number': 4, 'subreddit': 'gameofthrones', 'tomato': '98%'}
{'air_date': '2014-05-11', 'episode_number': 6, 'name': 'The Laws of Gods and Men', 'season_number': 4, 'subreddit': 'gameofthrones', 'tomato': '96%'}
2
3
{'air_date': '2014-05-18', 'episode_number': 7, 'name': 'Mockingbird', 'season_number': 4, 'subreddit': 'gameofthrones', 'tomato': '100%'}
{'air_date': '2014-06-01', 'episode_number': 8, 'name': 'The Mountain and the Viper', 'season_number': 4, 'subreddit': 'gameofthrone

{'air_date': '2015-03-31', 'episode_number': 17, 'name': 'Tricksters', 'season_number': 1, 'subreddit': 'flashtv', 'tomato': '95%'}
{'air_date': '2015-04-14', 'episode_number': 18, 'name': 'All-Star Team Up', 'season_number': 1, 'subreddit': 'flashtv', 'tomato': '77%'}
{'air_date': '2015-04-21', 'episode_number': 19, 'name': 'Who Is Harrison Wells?', 'season_number': 1, 'subreddit': 'flashtv', 'tomato': '80%'}
2
{'air_date': '2015-04-28', 'episode_number': 20, 'name': 'The Trap', 'season_number': 1, 'subreddit': 'flashtv', 'tomato': '100%'}
{'air_date': '2015-05-05', 'episode_number': 21, 'name': 'Grodd Lives', 'season_number': 1, 'subreddit': 'flashtv', 'tomato': '100%'}
{'air_date': '2015-05-12', 'episode_number': 22, 'name': 'Rogue Air', 'season_number': 1, 'subreddit': 'flashtv', 'tomato': '86%'}
2
{'air_date': '2015-05-19', 'episode_number': 23, 'name': 'Fast Enough', 'season_number': 1, 'subreddit': 'flashtv', 'tomato': '100%'}
{'air_date': '2015-10-06', 'episode_number': 1, 'nam

{'air_date': '2017-11-28', 'episode_number': 8, 'name': 'Crisis on Earth-X (III)', 'season_number': 4, 'subreddit': 'flashtv', 'tomato': '89%'}
{'air_date': '2017-12-05', 'episode_number': 9, 'name': "Don't Run", 'season_number': 4, 'subreddit': 'flashtv', 'tomato': '100%'}
{'air_date': '2018-01-16', 'episode_number': 10, 'name': 'The Trial of The Flash', 'season_number': 4, 'subreddit': 'flashtv', 'tomato': '63%'}
{'air_date': '2018-01-23', 'episode_number': 11, 'name': 'The Elongated Knight Rises', 'season_number': 4, 'subreddit': 'flashtv', 'tomato': '89%'}
{'air_date': '2018-01-30', 'episode_number': 12, 'name': 'Honey, I Shrunk Team Flash', 'season_number': 4, 'subreddit': 'flashtv', 'tomato': '75%'}
{'air_date': '2018-02-06', 'episode_number': 13, 'name': 'True Colors', 'season_number': 4, 'subreddit': 'flashtv', 'tomato': '43%'}
{'air_date': '2018-02-27', 'episode_number': 14, 'name': 'Subject 9', 'season_number': 4, 'subreddit': 'flashtv', 'tomato': '71%'}
{'air_date': '2018-03

In [17]:
db.posts.count_documents({})

22974

In [496]:
db.comments.drop()
#'subreddit': 'westworld'
for episode in tqdm(list(db.tv.find({} ,{'_id':0, 'subreddit':1,
                                    'air_date': 1,
                                    'episode_number' : 1,
                                    'season_number': 1,
                                    'name': 1,
                                    'tomato': 1,
                                   }))):
#     print(episode)
    time.sleep(.1)
    comments = get_reddit_posts(episode['subreddit'],episode['air_date'],endpoint = 'comment',limit = 200)
    if comments:
        [ comment.update({'episode': episode}) for comment in comments ]
        db.comments.insert_many(comments)

1
1
2
1



In [18]:
db.comments.count_documents({})

46800

In [19]:
list(db.comments.find({'subreddit': 'gameofthrones'},{"episode":1})) # ,{'_id' : 0, 'title':1, 'selftext' : 1}

[{'_id': ObjectId('5ec551189dbdf5cbda7b6aa2'),
  'episode': {'air_date': '2011-04-17',
   'episode_number': 1,
   'name': 'Winter Is Coming',
   'season_number': 1,
   'subreddit': 'gameofthrones',
   'tomato': '100%'}},
 {'_id': ObjectId('5ec551189dbdf5cbda7b6aa3'),
  'episode': {'air_date': '2011-04-17',
   'episode_number': 1,
   'name': 'Winter Is Coming',
   'season_number': 1,
   'subreddit': 'gameofthrones',
   'tomato': '100%'}},
 {'_id': ObjectId('5ec551189dbdf5cbda7b6aa4'),
  'episode': {'air_date': '2011-04-17',
   'episode_number': 1,
   'name': 'Winter Is Coming',
   'season_number': 1,
   'subreddit': 'gameofthrones',
   'tomato': '100%'}},
 {'_id': ObjectId('5ec551189dbdf5cbda7b6aa5'),
  'episode': {'air_date': '2011-04-17',
   'episode_number': 1,
   'name': 'Winter Is Coming',
   'season_number': 1,
   'subreddit': 'gameofthrones',
   'tomato': '100%'}},
 {'_id': ObjectId('5ec551189dbdf5cbda7b6aa6'),
  'episode': {'air_date': '2011-04-17',
   'episode_number': 1,
   'n

In [23]:
url = f'https://api.themoviedb.org/3/tv/63247/credits?api_key={API_KEY}'
r = requests.get(url)
json.loads(r.text)['cast'][0]

{'character': 'Dolores Abernathy',
 'credit_id': '55a610acc3a3682bb4000be0',
 'id': 38940,
 'name': 'Evan Rachel Wood',
 'gender': 1,
 'profile_path': '/g6XBBmsEd9fqP0gc4RuHsX0MXNl.jpg',
 'order': 0}